In [15]:
pip install redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 15.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import redis

In [5]:
import boto3
import json
import base64
import time
import re
from botocore.exceptions import ClientError
import logging
import redis

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

bedrock = boto3.client(
    service_name='bedrock', 
    region_name='us-east-1'
)

brt = boto3.client(service_name='bedrock-runtime')

##Processing data to fetch role and content
def data_preprocessing(data):
    #data = json_data['Records'][0]["body"]
    convo = ""
    content = json.loads(data["body"])["body"]["transcript"][0]["Content"]
    role = json.loads(data["body"])["body"]["transcript"][0]["ParticipantRole"]
    convo = convo + role + " : " + content
    
    return convo

def data_postprocessing(data):
    result = ""
    start_index = data.find("{")
    
    if start_index == -1:
        data1 = "{" + data
    else:
        data1 = data
    
    start_index_final = data.find("{")    
    print(f'start_index:{start_index_final}')
    
    end_char_indices = [i.start() for i in re.finditer("}",data1)]
    print(f'end_indices:{end_char_indices}')
    end_index = end_char_indices[len(end_char_indices)-1]
    print(f'end_index:{end_index}')
    if end_index == len(data1)-1:
        result = data1[start_index_final:]
    else:
        result = data1[start_index_final:end_index+1]
    print(f'result:{result}')
    
    return result
    
def summarisation_prompt_generator(context):
    prompt_llama = f"""
Instruction: "Summarise this call transcript between a patient and an agent and provide it in a precise paragraph : " :

{context}.
             
Response :  
    """
    return prompt_llama
    
#Defining function to summarize context
def load_llama2(bedrock_runtime , prompt , temp , top_p):
    try:
        body = {
            "prompt" : prompt,
            "temperature" : temp,
            "top_p" : top_p,
            "max_gen_len" : 1000
            }

        response = bedrock_runtime.invoke_model(
            modelId="meta.llama2-13b-chat-v1", body=json.dumps(body)
        )

        response_body = json.loads(response["body"].read())
        completion = response_body["generation"]

        return completion

    except ClientError:
        logging.error("Couldn't invoke Llama 2")
        raise
    
def SQS_Publisher(json_data):
    try:
        sqs = boto3.client('sqs',region_name='us-east-1')
        # Define the SQS queue URL
        queue_url = 'https://sqs.us-east-1.amazonaws.com/383299343633/ch-agent-assist-sqs'
        response = sqs.send_message(QueueUrl= queue_url,MessageBody=json.dumps(json_data) )
    except Exception as e:
        print(e)
        
def SQS_data_postprocessing(event,data):
    json_response = {
            "stream": "SUMMARY_DATA",
            "streamConnectionId": json.loads(event["Records"][0]['body'])["streamConnectionId"],
             "body": {
                 "transactionId": "f830e890-3ff2-4fdc-a08e-dd9b78a2dc28",
                  "contactId": json.loads(event["Records"][0]['body'])["streamConnectionId"],
            "form_data": data,
                     }  
            }
    return json_response
    
def lambda_handler(event,context):
    final_transcript = ""
    #print(f'lambda_handler--Received_input : {event}.')
    for i in range(len(event["Records"])):
        final_transcript += "\n" + data_preprocessing(event["Records"][i])
    #print(f'lambda_handler--output_after_preprocessing_data : {final_transcript}.')
    
    prompt_summary = summarisation_prompt_generator(final_transcript)
    #print(f'lambda_handler--generated_prompt : {prompt_summary}.')
    summary_data = load_llama2(bedrock_runtime,prompt_summary,0.5,0.9)
    print(f'lambda_handler--output_from_the_llm_model : {summary_data}.')
    sqs_data = SQS_data_postprocessing(event,summary_data)
    #print(f'lambda_handler--data_to_be_sent_to_SQS_after_postprocessing_json_output_of_model : {sqs_data}.')
    ##SQS_Publisher(sqs_data)
    ##print(f'lambda_handler--data_sent_to_sqs_sucessfully.')
    #return {"statusCode": 200,"body": "Data Sent to SQS Sucessfully"}

In [1]:
import boto3
import json
import base64
import time
import re
from botocore.exceptions import ClientError
import logging
import redis
import os

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

bedrock = boto3.client(
    service_name='bedrock', 
    region_name='us-east-1'
)

brt = boto3.client(service_name='bedrock-runtime')

# Initialize Redis client
redis_host = 'ch-agent-assist-redis-cluster-disabled-ro.tjyhst.ng.0001.use1.cache.amazonaws.com'
redis_port = 6379 # Default Redis port is 6379
redis_password = None  # None if no password
redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)

##Processing data to fetch role and content
def data_preprocessing(data):
        
    json_str1 = '[' + data
    json_str1 += ']'
    json_str2 = json_str1.strip()

    try:
       # Parse the JSON string as a list of dictionaries
       data_list = json.loads(json_str2)
       convo = ""
       # Iterate through each item in the list and print desired information
       for data in data_list:
            for transcript in data['transcript']:
                participant_role = transcript['ParticipantRole']
                content = transcript['Content']
                convo += participant_role + " : " + content + "\n"
            #print(f'{participant_role} : {content}')
                
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        
    return convo

def data_postprocessing(data):
    result = ""
    start_index = data.find("{")
    
    if start_index == -1:
        data1 = "{" + data
    else:
        data1 = data
    
    start_index_final = data.find("{")    
    print(f'start_index:{start_index_final}')
    
    end_char_indices = [i.start() for i in re.finditer("}",data1)]
    print(f'end_indices:{end_char_indices}')
    end_index = end_char_indices[len(end_char_indices)-1]
    print(f'end_index:{end_index}')
    if end_index == len(data1)-1:
        result = data1[start_index_final:]
    else:
        result = data1[start_index_final:end_index+1]
    print(f'result:{result}')
    
    return result
    
def summarisation_prompt_generator(context):
    prompt_llama = f"""
Instruction: "Summarise this call transcript between a patient and an agent and provide it in a precise paragraph : " :

{context}.
             
Response :  
    """
    return prompt_llama
    
#Defining function to summarize context
def load_llama2(bedrock_runtime , prompt , temp , top_p):
    try:
        body = {
            "prompt" : prompt,
            "temperature" : temp,
            "top_p" : top_p,
            "max_gen_len" : 1000
            }

        response = bedrock_runtime.invoke_model(
            modelId="meta.llama2-13b-chat-v1", body=json.dumps(body)
        )

        response_body = json.loads(response["body"].read())
        completion = response_body["generation"]

        return completion

    except ClientError:
        logging.error("Couldn't invoke Llama 2")
        raise
    
def SQS_Publisher(json_data):
    try:
        sqs = boto3.client('sqs',region_name='us-east-1')
        # Define the SQS queue URL
        queue_url = 'https://sqs.us-east-1.amazonaws.com/383299343633/ch-agent-assist-sqs'
        response = sqs.send_message(QueueUrl= queue_url,MessageBody=json.dumps(json_data) )
    except Exception as e:
        print(e)
        
def SQS_data_postprocessing(event,data):
    json_response = {
            "stream": "SUMMARY_DATA",
            "streamConnectionId": json.loads(event["Records"][0]['body'])["streamConnectionId"],
             "body": {
                 "transactionId": "f830e890-3ff2-4fdc-a08e-dd9b78a2dc28",
                  "contactId": json.loads(event["Records"][0]['body'])["streamConnectionId"],
            "form_data": data,
                     }  
            }
    return json_response
    
def lambda_handler(event,context):
    #print(event) 
    # Initialize Redis client
    #redis_host = 'ch-agent-assist-redis-cluster-disabled-ro.tjyhst.ng.0001.use1.cache.amazonaws.com:6379'
    #redis_port =  6379  # Default Redis port is 6379
    #redis_password = None  # None if no password
    #redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)
    key_to_read = json.loads(event["Records"][0]['body'])["streamConnectionId"] #event.get('conversation_id')
    print(key_to_read)
    # Check if the key exists and read the value from Redis
    if key_to_read and redis_client.exists(key_to_read):
        value = redis_client.get(key_to_read)
        print(value) #.split("contactId"))
    else:
        print('Key not found')
        
    #final_transcript = ""
    #print(f'lambda_handler--Received_input : {event}.')
    #for i in range(len(event["Records"])):
        #final_transcript += "\n" + data_preprocessing(event["Records"][i])
    #print(f'lambda_handler--output_after_preprocessing_data : {final_transcript}.')
    final_transcript = data_preprocessing(value)
    print(final_transcript)
    prompt_summary = summarisation_prompt_generator(final_transcript)
    print(f'lambda_handler--generated_prompt : {prompt_summary}.')
    summary_data = load_llama2(bedrock_runtime,prompt_summary,0.5,0.9)
    print(f'lambda_handler--output_from_the_llm_model : {summary_data}.')
    sqs_data = SQS_data_postprocessing(event,summary_data)
    print(f'lambda_handler--data_to_be_sent_to_SQS_after_postprocessing_json_output_of_model : {sqs_data}.')
    SQS_Publisher(sqs_data)
    print(f'lambda_handler--data_sent_to_sqs_sucessfully.')
    return {"statusCode": 200,"body": "Data Sent to SQS Sucessfully"}